In [1]:
import pandas as pd
import geopandas as gpd

Vygenerovanú query treba zbehnúť v Overpass Turbo API (https://overpass-turbo.eu) a exportovať ako GEOJSON a následne uložiť ako ./source-files/sportoviska.geojson

In [2]:
def generate_query_for_sports(sports, out_center=False):
    """
    Funkcia vygeneruje dotaz použitelný pre Overpass Turbo, ktorý načíta športoviská podľa zadaných športov.

    Argumenty:
    sports (list of str): Zoznam športov, ktoré chceme zahrnúť do dotazu.
    out_center (bool): Ak je True, dotaz pre každý OSM objekt vypočíta reprezentatívny bod.

    Návratová hodnota:
    str: Dotaz vo formáte Overpass QL, ktorý načíta športoviská podľa zadaných kritérií.

    Príklad použitia:
    generate_query_for_sports(["soccer", "swimming"], out_center=True)
    """
    
    # Prefix pre dotaz obsahujúci základné nastavenia a oblasť hľadania
    prefix = """
[out:json][timeout:120];
{{geocodeArea:Slovakia}}->.searchArea;
"""
    
    # Vygenerovanie dotazu pre zadané športy
    query = f"""(
  nwr["sport"]["sport"~"^({'|'.join(sports)})$"](area.searchArea);
  nwr["leisure"="swimming_pool"](area.searchArea);
  nwr["leisure"="water_park"](area.searchArea);
  nwr["leisure"="ice_rink"](area.searchArea);
  nwr["leisure"="pitch"]["sport"!~".*"](area.searchArea);
  nwr["leisure"="sports_centre"]["sport"!~".*"](area.searchArea);
  nwr["leisure"="stadium"]["sport"!~".*"](area.searchArea);
  nwr["building"="stadium"](area.searchArea);
);
out geom;
{"out center;" if out_center else ''}
out meta;
out skel qt;
"""
    
    # Vrátenie výsledného dotazu
    return prefix + query

# Príklad použitia funkcie
sports = ["soccer", "swimming", "ice_hockey", "volleyball", "beachvolleyball", "ice_skating", "basketball", "tennis", "multi", "street_hockey", "field_hockey", "hockey"]


sports = ["soccer", "swimming", "ice_hockey", "volleyball", "beachvolleyball", "ice_skating", "basketball", "tennis", "multi", "street_hockey", "field_hockey", "hockey"]
print(generate_query_for_sports(sports))


[out:json][timeout:120];
{{geocodeArea:Slovakia}}->.searchArea;
(
  nwr["sport"]["sport"~"^(soccer|swimming|ice_hockey|volleyball|beachvolleyball|ice_skating|basketball|tennis|multi|street_hockey|field_hockey|hockey)$"](area.searchArea);
  nwr["leisure"="swimming_pool"](area.searchArea);
  nwr["leisure"="water_park"](area.searchArea);
  nwr["leisure"="ice_rink"](area.searchArea);
  nwr["leisure"="pitch"]["sport"!~".*"](area.searchArea);
  nwr["leisure"="sports_centre"]["sport"!~".*"](area.searchArea);
  nwr["leisure"="stadium"]["sport"!~".*"](area.searchArea);
  nwr["building"="stadium"](area.searchArea);
);
out geom;

out meta;
out skel qt;



ČISTENIE A PREPROCESSING DÁT 

In [3]:
# Nacitava sa zdrojovy subor dat, ktory dostavame po zbehnuti vyssie vygenerovanej query v OverpassTurbo a exporte vo formate geojson
sportoviska_load = gpd.read_file(r'./source-files/sportoviska.geojson')
sportoviska_load.head()

,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,wikidata,wikimedia_commons,wikipedia,@timestamp,@version,@changeset,@user,@uid,@relations,geometry
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,None,None,None,2023-01-21 10:47:11+00:00,4.0,131536626.0,Klerik7,7169223.0,None,"POLYGON ((17.59039 48.37420, 17.59050 48.37399..."
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,Q392323,None,sk:Štadión MŠK Žilina,2022-12-28 09:24:36+00:00,6.0,130590529.0,Dodko,1944033.0,None,"POLYGON ((18.74464 49.22971, 18.74426 49.22980..."
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,None,None,None,2023-10-18 16:03:46+00:00,4.0,142770571.0,Klerik7,7169223.0,None,"POLYGON ((17.29396 48.19578, 17.29334 48.19531..."
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,None,None,None,2022-12-19 16:51:22+00:00,4.0,130256858.0,platlas23,115468.0,None,"POLYGON ((17.21628 48.21058, 17.21546 48.21031..."
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,None,None,None,2022-12-26 20:17:10+00:00,13.0,130529562.0,*Martin*,4815.0,None,"POLYGON ((20.01019 48.34573, 20.01123 48.34580..."


In [4]:
print(len(sportoviska_load))
sportoviska_load.dtypes


12576


id                     object
@id                    object
FIXME                  object
abandoned              object
abandoned:leisure      object
                       ...   
@changeset            float64
@user                  object
@uid                  float64
@relations             object
geometry             geometry
Length: 201, dtype: object

In [5]:
# Vyhodnoti ci query skutocne zachytila sportoviska urcene na sporty futbal, basketbal, tenis, plavanie, hokej, volejbal (pripadne im podobne)
# V pripade ak sa v atribute sport nenachadza aspon jeden z vyssie vymenovanych sportov, tak sa objekt oznaci, ze neobsahuje validny sport.
# Vysledok tejto bunky su dva nove stlpce: binarny priznak validne_sporty, a mnozina dostupnych sportov na sportovisku pod atributom vsetky_sporty

# Vytvorenie množiny športov zo zoznamu 'sports'
mnozina_sportov = set(sports)

# Vytvorenie kópie pôvodného DataFrame
sportoviska_prep = sportoviska_load.copy()

# Vytvorenie nového stĺpca 'vsetky_sporty', ktorý obsahuje množinu športov zadaných v stĺpci 'sport'
sportoviska_prep["vsetky_sporty"] = sportoviska_prep['sport'].apply(lambda sport: set(sport.split(';')) if sport else None)

# Aktualizácia stĺpca 'vsetky_sporty' spojením so stĺpcom 'sport_1'
sportoviska_prep["vsetky_sporty"] = sportoviska_prep.apply(
    lambda sportovisko: sportovisko["vsetky_sporty"].union({sportovisko['sport_1']}) if sportovisko["vsetky_sporty"] else None, 
    axis=1
)

def vyhodnot_sporty(sportovisko):
    """
    Funkcia vyhodnotí, či športovisko má aspoň jeden šport, ktorý je v množine zadaných športov.
    Ak áno, vráti 1, inak 0.
    
    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.
    
    Návratová hodnota:
    int: 1 ak športovisko má aspoň jeden šport v množine zadaných športov, inak 0.
    """
    if sportovisko["vsetky_sporty"]:
        if len(sportovisko["vsetky_sporty"].intersection(mnozina_sportov)) > 0:
            return 1
    elif sportovisko["vsetky_sporty"] is None:
        return 1
    return 0

# Vytvorenie nového stĺpca 'validne_sporty', ktorý vyhodnotí športoviská podľa funkcie 'vyhodnot_sporty'
sportoviska_prep["validne_sporty"] = sportoviska_prep.apply(lambda sportovisko: vyhodnot_sporty(sportovisko), axis=1)

def remove_none_from_set(s):
    """
    Funkcia odstráni prvok None z množiny.
    
    Argumenty:
    s (set): Množina, z ktorej sa má odstrániť prvok None.
    
    Návratová hodnota:
    set: Množina bez prvku None.
    """
    if s is not None:
        s.discard(None)
        return s
    return set()

# Odstránenie prvkov None z množiny v stĺpci 'vsetky_sporty'
sportoviska_prep["vsetky_sporty"] = sportoviska_prep['vsetky_sporty'].apply(remove_none_from_set)

# Vytvorenie nového stĺpca 'vsetky_sporty_str', ktorý obsahuje športy ako reťazec oddelený bodkočiarkou
sportoviska_prep["vsetky_sporty_str"] = sportoviska_prep['vsetky_sporty'].apply(lambda x: ";".join(list(x)))

# Zobrazenie prvých riadkov upraveného DataFrame
sportoviska_prep.head()

,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,@timestamp,@version,@changeset,@user,@uid,@relations,geometry,vsetky_sporty,validne_sporty,vsetky_sporty_str
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,2023-01-21 10:47:11+00:00,4.0,131536626.0,Klerik7,7169223.0,None,"POLYGON ((17.59039 48.37420, 17.59050 48.37399...",{},1,
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,2022-12-28 09:24:36+00:00,6.0,130590529.0,Dodko,1944033.0,None,"POLYGON ((18.74464 49.22971, 18.74426 49.22980...",{},1,
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,2023-10-18 16:03:46+00:00,4.0,142770571.0,Klerik7,7169223.0,None,"POLYGON ((17.29396 48.19578, 17.29334 48.19531...",{soccer},1,soccer
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,2022-12-19 16:51:22+00:00,4.0,130256858.0,platlas23,115468.0,None,"POLYGON ((17.21628 48.21058, 17.21546 48.21031...",{soccer},1,soccer
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,2022-12-26 20:17:10+00:00,13.0,130529562.0,*Martin*,4815.0,None,"POLYGON ((20.01019 48.34573, 20.01123 48.34580...",{swimming},1,swimming


In [6]:
# Ponechá len športoviská s validným športom
sportoviska_filtered = sportoviska_prep[sportoviska_prep["validne_sporty"] == 1]
print(len(sportoviska_filtered))
sportoviska_filtered.head()

12574


,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,@timestamp,@version,@changeset,@user,@uid,@relations,geometry,vsetky_sporty,validne_sporty,vsetky_sporty_str
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,2023-01-21 10:47:11+00:00,4.0,131536626.0,Klerik7,7169223.0,None,"POLYGON ((17.59039 48.37420, 17.59050 48.37399...",{},1,
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,2022-12-28 09:24:36+00:00,6.0,130590529.0,Dodko,1944033.0,None,"POLYGON ((18.74464 49.22971, 18.74426 49.22980...",{},1,
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,2023-10-18 16:03:46+00:00,4.0,142770571.0,Klerik7,7169223.0,None,"POLYGON ((17.29396 48.19578, 17.29334 48.19531...",{soccer},1,soccer
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,2022-12-19 16:51:22+00:00,4.0,130256858.0,platlas23,115468.0,None,"POLYGON ((17.21628 48.21058, 17.21546 48.21031...",{soccer},1,soccer
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,2022-12-26 20:17:10+00:00,13.0,130529562.0,*Martin*,4815.0,None,"POLYGON ((20.01019 48.34573, 20.01123 48.34580...",{swimming},1,swimming


In [7]:
from shapely.geometry import Polygon, Point, MultiPolygon


def futbal(sportovisko):
    """
    Funkcia vyhodnotí, či sa na danom športovisku dá hrať futbal. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa futbal (soccer alebo football), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"soccer", "basketball"}})
    >>> futbal(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"basketball", "tennis"}})
    >>> futbal(sportovisko)
    0
    """
    if "soccer" in sportovisko["vsetky_sporty"] or "football" in sportovisko["vsetky_sporty"]:
        return 1
    return 0

def plavanie(sportovisko):
    """
    Funkcia vyhodnotí, či sa na danom športovisku dá plávať. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa plávanie (swimming), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"swimming", "basketball"}})
    >>> plavanie(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"basketball", "tennis"}})
    >>> plavanie(sportovisko)
    0
    """
    if "swimming" in sportovisko["vsetky_sporty"]:
        return 1
    return 0

def basketbal(sportovisko):
    """
    Funkcia vyhodnotí, či sa na danom športovisku dá hrať basketbal. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa basketbal (basketball), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"basketball", "tennis"}})
    >>> basketbal(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"swimming", "tennis"}})
    >>> basketbal(sportovisko)
    0
    """
    if "basketball" in sportovisko["vsetky_sporty"]:
        return 1
    return 0

def tenis(sportovisko):
    """
    Funkcia vyhodnotí, či sa na danom športovisku dá hrať tenis. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa tenis (tennis), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"tennis", "basketball"}})
    >>> tenis(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"swimming", "basketball"}})
    >>> tenis(sportovisko)
    0
    """
    if "tennis" in sportovisko["vsetky_sporty"]:
        return 1
    return 0

def volejbal(sportovisko):
    """
    Funkcia vyhodnotí, či sa na danom športovisku dá hrať forma volejbalu. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa volejbal (volleyball) alebo plážový volejbal (beachvolleyball), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"volleyball", "basketball"}})
    >>> volejbal(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"swimming", "basketball"}})
    >>> volejbal(sportovisko)
    0
    """
    if "volleyball" in sportovisko["vsetky_sporty"] or "beachvolleyball" in sportovisko["vsetky_sporty"]:
        return 1
    return 0

def hokej(sportovisko):
    """
    Funkcia vyhodnotí, či sa na danom športovisku dá hrať hokej. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa hokej (ice_hockey) alebo korčuľovanie (ice_skating), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"ice_hockey", "basketball"}})
    >>> hokej(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"swimming", "basketball"}})
    >>> hokej(sportovisko)
    0
    """
    if "ice_hockey" in sportovisko["vsetky_sporty"] or "ice_skating" in sportovisko["vsetky_sporty"]:
        return 1
    return 0

def multi(sportovisko):
    """
    Funkcia vyhodnotí, či je športovisko viacúčelové. Ak áno, vráti 1, inak 0.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko, ktorý obsahuje stĺpec 'vsetky_sporty'.

    Návratová hodnota:
    int: 1 ak športovisko zahŕňa viacúčelové športy (multi), inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"vsetky_sporty": {"multi", "basketball"}})
    >>> multi(sportovisko)
    1

    >>> sportovisko = pd.Series({"vsetky_sporty": {"swimming", "basketball"}})
    >>> multi(sportovisko)
    0
    """
    if "multi" in sportovisko["vsetky_sporty"]:
        return 1
    return 0



def reprezentativny_bod(geometry):
    """
    Funkcia priradí objektu reprezentatívny bod podľa geometrického typu daného objektu.

    Argumenty:
    geometry (shapely.geometry): Geometrický objekt, ktorý môže byť typu Polygon, MultiPolygon alebo Point.

    Návratová hodnota:
    shapely.geometry.Point alebo None: Reprezentatívny bod geometrie. Ak je geometria typu Polygon alebo MultiPolygon, vráti centroid. Ak je geometria typu Point, vráti samotný bod. V ostatných prípadoch vráti None.

    Príklad použitia:
    >>> from shapely.geometry import Point, Polygon
    >>> polygon = Polygon([(0, 0), (1, 1), (1, 0)])
    >>> reprezentativny_bod(polygon)
    <shapely.geometry.point.Point object at 0x...>

    >>> point = Point(1, 1)
    >>> reprezentativny_bod(point)
    <shapely.geometry.point.Point object at 0x...>
    """
    # Priradí objektu reprezentatívny bod podľa geometrického typu
    if isinstance(geometry, (Polygon, MultiPolygon)):
        return geometry.centroid
    elif isinstance(geometry, Point):
        return geometry
    else:
        return None

sportoviska_sporty = sportoviska_filtered.copy()
sportoviska_sporty['futbal'] = sportoviska_filtered.apply(futbal, axis = 1)
sportoviska_sporty['plavanie'] = sportoviska_filtered.apply(plavanie, axis = 1)
sportoviska_sporty['basketbal'] = sportoviska_filtered.apply(basketbal, axis = 1)
sportoviska_sporty['volejbal'] = sportoviska_filtered.apply(volejbal, axis = 1)
sportoviska_sporty['tenis'] = sportoviska_filtered.apply(tenis, axis = 1)
sportoviska_sporty['hokej'] = sportoviska_filtered.apply(hokej, axis = 1)
sportoviska_sporty['multi'] = sportoviska_filtered.apply(multi, axis = 1)

sportoviska_sporty = sportoviska_sporty[~sportoviska_sporty['geometry'].geom_type.isin(['LineString'])] # Odstrani sportoviska linioveho charakteru

sportoviska_sporty['suradnice'] = sportoviska_sporty['geometry'].apply(reprezentativny_bod)
sportoviska_sporty.head()

,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,validne_sporty,vsetky_sporty_str,futbal,plavanie,basketbal,volejbal,tenis,hokej,multi,suradnice
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,1,,0,0,0,0,0,0,0,POINT (17.59182 48.37372)
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,1,,0,0,0,0,0,0,0,POINT (18.74481 49.22913)
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,1,soccer,1,0,0,0,0,0,0,POINT (17.29423 48.19521)
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,1,soccer,1,0,0,0,0,0,0,POINT (17.21558 48.20993)
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,1,swimming,0,1,0,0,0,0,0,POINT (20.01876 48.34455)


In [8]:
data = sportoviska_sporty.copy()
percent_not_null = data.notnull().mean() * 100
data_filtered = data.loc[:, percent_not_null > 0] # Odstrani atribnuty, ktore maju 0% hustotu
data_filtered = data_filtered[data_filtered['club'] != 'sport'] # Odstrani objekty reprezentujuce korespondence adresy sportovych klubov ako organizacii
data_filtered = data_filtered[data_filtered['disused:club'] != 'sport'] # odstrani objekty reprezentujuce zaniknute korespondence adresy sportovych klubov ako organizacii

# Zoznam sportovych obchodov, ktore query zachytila. Bolo ich mozne rozoznat podla nazvu.
shops = [
    'node/6422021726',
    'node/6532877640',
    'node/7887992405',
    'node/10863706835',
    'node/10863706836',
    'node/10903359971',
    'way/61612988',
    'way/314937634',
    'way/1013356063',
    'node/5112613101'

]

data_filtered = data_filtered[~data_filtered['id'].isin(shops)] # Odstranuje zo suboru dat objekty reprezentujuce sportove obchody

print(data_filtered.shape) 
data_filtered.head()

(12424, 212)


,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,validne_sporty,vsetky_sporty_str,futbal,plavanie,basketbal,volejbal,tenis,hokej,multi,suradnice
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,1,,0,0,0,0,0,0,0,POINT (17.59182 48.37372)
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,1,,0,0,0,0,0,0,0,POINT (18.74481 49.22913)
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,1,soccer,1,0,0,0,0,0,0,POINT (17.29423 48.19521)
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,1,soccer,1,0,0,0,0,0,0,POINT (17.21558 48.20993)
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,1,swimming,0,1,0,0,0,0,0,POINT (20.01876 48.34455)


PRIPRAVA NA KLASIFIKACIU

In [9]:
classified_data = data_filtered.copy()

In [10]:
# Spaja atributy s rovnakym vyznamom do jedneho noveho atributu a zahadzuje stare

from functools import reduce

try:
    # Kombinácia stĺpcov 'contact:email' a 'email' do jedného stĺpca 'kontakt_email'
    cols = ['contact:email', 'email']
    classified_data['kontakt_email'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'contact:facebook' a 'facebook' do jedného stĺpca 'kontakt_facebook'
    cols = ['contact:facebook', 'facebook']
    classified_data['kontakt_facebook'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'contact:phone', 'phone' a 'mobile' do jedného stĺpca 'kontakt_telefon'
    cols = ['contact:phone', 'phone', 'mobile']
    classified_data['kontakt_telefon'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'website', 'contact:website' a 'url' do jedného stĺpca 'kontakt_webstranka'
    cols = ['website', 'contact:website', 'url']
    classified_data['kontakt_webstranka'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'indoor' a 'covered' do jedného stĺpca 'indoor'
    cols = ['indoor', 'covered']
    classified_data['indoor'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(['covered'], axis=1)

    # Kombinácia stĺpcov 'max_depth' a 'maxdepth' do jedného stĺpca 'max_hlbka'
    cols = ['max_depth', 'maxdepth']
    classified_data['max_hlbka'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'min_depth' a 'mindepth' do jedného stĺpca 'min_hlbka'
    cols = ['min_depth', 'mindepth']
    classified_data['min_hlbka'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'depth' a 'water:depth' do jedného stĺpca 'hlbka'
    cols = ['depth', 'water:depth']
    classified_data['hlbka'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'temperature', 'water:temperature' a 'water:temperature:min' do jedného stĺpca 'teplota_vody'
    cols = ['temperature', 'water:temperature', 'water:temperature:min']
    classified_data['teplota_vody'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia rôznych názvov do jedného stĺpca 'nazov'
    cols = ['name', 'official_name', 'name:sk', 'loc_name', 'alt_name', 'alt_name_1', 'alt_name_2']
    classified_data['nazov'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'water:max_temperature' a 'water:temperature:max' do jedného stĺpca 'max_teplota'
    cols = ['water:max_temperature', 'water:temperature:max']
    classified_data['max_teplota'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpca 'capacity' do stĺpca 'kapacita_divaci'
    cols = ['capacity']  # 'capacity:persons' je komentovaný
    classified_data['kapacita_divaci'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'charge' a 'fee' do jedného stĺpca 'poplatok'
    cols = ['charge', 'fee']
    classified_data['poplatok'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(cols, axis=1)

    # Kombinácia stĺpcov 'building' a 'building:part' do jedného stĺpca 'building'
    cols = ['building', 'building:part']
    classified_data['building'] = reduce(lambda x, y: x.combine_first(y), [classified_data[col] for col in cols])
    classified_data = classified_data.drop(['building:part'], axis=1)

    def location(sportovisko):
        """
        Vyhodnocuje umiestnenie športoviska na základe iných tagov.

        Argumenty:
        sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

        Návratová hodnota:
        pd.Series: Upravený riadok s novým stĺpcom 'location'.
        """
        if sportovisko['indoor'] == 'yes' or sportovisko['building'] not in [None, 'no']:
            sportovisko['location'] = 'indoor'
        elif sportovisko['indoor'] == 'no':
            sportovisko['location'] = 'outdoor'

        if sportovisko['outdoor'] == 'yes':
            sportovisko['location'] = 'outdoor'
        elif sportovisko['outdoor'] == 'no':
            sportovisko['location'] = 'indoor'

        return sportovisko

    def leisure_pitch_playground(sportovisko):
        """
        Zmení hodnotu stĺpca 'leisure' z 'playground' na 'pitch', ak je hodnota 'playground'.

        Argumenty:
        sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

        Návratová hodnota:
        pd.Series: Upravený riadok s prípadne zmenenou hodnotou stĺpca 'leisure'.
        """
        if sportovisko['leisure'] == 'playground':
            sportovisko['leisure'] = 'pitch'
        return sportovisko

    # Aplikácia funkcie 'location' na celý DataFrame
    classified_data = classified_data.apply(location, axis=1)
    # Odstránenie stĺpcov 'indoor' a 'outdoor'
    cols = ['indoor', 'outdoor']
    classified_data = classified_data.drop(cols, axis=1)
    # Aplikácia funkcie 'leisure_pitch_playground' na celý DataFrame
    classified_data = classified_data.apply(leisure_pitch_playground, axis=1)
    # Nastavenie hodnoty 'leisure' na 'stadium', ak je 'building' rovné 'stadium'
    classified_data.loc[classified_data['building'] == 'stadium', 'leisure'] = 'stadium'

except Exception as e:
    print('chyba', e)

# Návrat spracovaného DataFrame
classified_data


,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,kontakt_telefon,kontakt_webstranka,max_hlbka,min_hlbka,hlbka,teplota_vody,nazov,max_teplota,kapacita_divaci,poplatok
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,None,https://www.mskzilina.sk/zobraz.asp?t=stadion,None,None,None,None,Štadión MŠK Žilina,None,None,None
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,FK Cepit Vajnory,None,None,None
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,None,None,None,None,None,None,Vodná nádrž Kurinec,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,node/11847211830,node/11847211830,None,None,None,private,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
12572,node/11847211831,node/11847211831,None,None,None,private,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
12573,node/11847211846,node/11847211846,None,None,None,private,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
12574,node/11847352225,node/11847352225,None,None,None,private,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [11]:
# DEDENIE VLASTNOSTI ZO SPORTOVISKA, KTORE OBSAHUJE DANE SPORTOVISKO ( DEDI SA Z NAJMENSIEHO MOZNEHO UTVARU, V KTOROM JE SPORTOVISKO OBSIAHNUTE)

# Špecifikujú sa stĺpce, na ktorých chceme previesť dedenie
columns_to_inherit = ['location']

# Pre každý geometrický objekt sa vypočíta jeho plocha
classified_data['area'] = classified_data['geometry'].apply(lambda x: x.area)

# Pomocou operácie Spatial Join sa spraví self join tabulky s predikátom 'contains' (obsahuje).
# Výsledkom budú dvojice športovísk, kde jedno obsahuje druhé
join = gpd.sjoin(classified_data, classified_data, how='left', predicate='contains')

# Odstráni športoviská, ktoré sa pripojili samy na seba
join = join[join['id_left'] != join['id_right']]

# Pridá stĺpec 'area_right' s hodnotou plochy pravého športoviska
join = join.merge(classified_data[['area']], left_on='index_right', right_index=True, suffixes=('', '_right'))

# Pre športoviská identifikuje najmenšiu obsahujúcu geometriu
smallest_container = join.groupby('index_right').apply(lambda x: x.nsmallest(1, 'area_right'))

# Funkcia na dedenie hodnôt určeného stĺpca
def inherit_values(sportovisko):
    """
    Priradí hodnotu určeného stĺpca športovisku (ktorý má tento stĺpec prázdny)
    zo športoviska, ktoré je najmenšie obsahujúce športovisko, ktoré dedí informáciu.
    
    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    Hodnota stĺpca po dedení, alebo pôvodná hodnota, ak dedenie nie je potrebné.
    """
    if pd.isna(sportovisko[column]):
        container_data = smallest_container[smallest_container.index_right == sportovisko.name]
        if not container_data.empty:
            return container_data.iloc[0][column + '_left']
    return sportovisko[column]

# Pre každý určený stĺpec vykoná dedenie hodnôt
for column in columns_to_inherit:
    classified_data[column] = classified_data.apply(inherit_values, axis=1)

# Odstráni dočasný stĺpec 'area'
classified_data.drop(['area'], axis=1, inplace=True)

KLASIFIKÁCIA ŠPORTOVÍSK

In [12]:
# rule based classifier v2
import re

def is_standalone(substring, string):
    """
    Zistí, či sa podreťazec v reťazci nachádza ako samostatné slovo.

    Argumenty:
    substring (str): Podreťazec, ktorý hľadáme.
    string (str): Reťazec, v ktorom hľadáme podreťazec.

    Návratová hodnota:
    bool: True, ak sa podreťazec nachádza ako samostatné slovo, inak False.

    Príklad použitia:
    >>> is_standalone('fk', 'FK Bratislava')
    True
    >>> is_standalone('fk', 'Bratislavafk')
    False
    """
    pattern = r'\b' + re.escape(substring) + r'\b'
    if re.search(pattern, string, re.IGNORECASE):
        return True
    else:
        return False
    
def rozoznaj_futbalove_skratky(name):
    """
    Rozozná v názve skratky, ktoré súvisia s futbalovými klubmi a implikujú možnosť hrania futbalu na danom športovisku.

    Argumenty:
    name (str): Názov športoviska.

    Návratová hodnota:
    bool: True, ak názov obsahuje futbalovú skratku, inak False.

    Príklad použitia:
    >>> rozoznaj_futbalove_skratky('FK Bratislava')
    True
    >>> rozoznaj_futbalove_skratky('Basketball Arena')
    False
    """
    if is_standalone('fk', name) or is_standalone('šk', name) or is_standalone('fc', name) or is_standalone('mfk', name):
        return True
    return False

kategorie = {
            'Nezaradené': 0000,
            'Multifunkčné ihrisko': 1110,
             "Volejbalové ihrisko": 1120,
             "Ihrisko na plážový volejbal": 1130,
             "Futbalové ihrisko": 1140,
             "Tenisový kurt": 1150,
             "Basketbalové ihrisko": 1160,
             "Hokejbalové ihrisko": 1170,
             "Ihrisko (nešpecifikované)": 1180,
             "Lúka": 1190,
             "Futbalový štadión": 1210,
             "Športový areál": 1220,
             "Štadión (nešpecifikované)": 1230,
             "Vonkajšia ľadová plocha": 1310,
             "Prírodný ľad": 1320,
             "Športová hala": 2110,
             "Tenisová hala": 2120,
             'Basketbalová hala': 2130,
             'Hokejbalová hala': 2140,
             'Futbalová hala': 2150,
             'Volejbalová hala': 2160,
             'Športové centrum': 2180,
             'Telocvičňa': 2170,
             "Zimný štadión": 2210,
             "Klzisko": 2220,
             "Hokejová hala": 2230,
             "Vnútorný bazén": 3110,
             "Plaváreň": 3120,
             "Vonkajší bazén": 3210,
             "Kúpalisko": 3220,
             "Jazero": 3230,
             "Pláž / Plavecká oblasť": 3240
             }

def kategorizuj_sportovisko(sportovisko):
    """
    Kategorizuje športovisko na základe názvu alebo hodnôt súvisiacich atribútov s daným typom športoviska.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: Kód kategórie športoviska.

    Príklad použitia:
    >>> sportovisko = pd.Series({"sport": "soccer", "leisure": "stadium", "vsetky_sporty": {"soccer"}, "nazov": "FK Bratislava"})
    >>> kategorizuj_sportovisko(sportovisko)
    1210
    """
    
    sport = sportovisko['sport'].lower() if sportovisko['sport'] else None
    leisure = sportovisko['leisure'].lower() if sportovisko['leisure'] else None
    sporty = sportovisko['vsetky_sporty']
    name = sportovisko['nazov']
    if not leisure:
        leisure = sportovisko['building']
    if leisure:
        leisure = leisure.lower()

    # Kategoria sa skryva v nazve
    if name:
        name = name.lower()

        for kategoria in kategorie.keys():
            if kategoria.lower() in name:
                return kategorie[kategoria]
        
        # osetrene nejake krajne pripady, kedy futbalovy stadion je nazvany podla Futbaloveho klubu
        if 'štadión' in name:
            if sport in ['soccer', 'football'] or 'futbal' in name or rozoznaj_futbalove_skratky(name): return kategorie["Futbalový štadión"]
            if 'zimný' in name: return kategorie['Zimný štadión']
        if leisure == 'stadium' and 'futbal' in name or rozoznaj_futbalove_skratky(name): return  kategorie["Futbalový štadión"]
        if 'hala' in name: 
            if sport == 'multi' or len(sporty) > 1:
                return kategorie['Športová hala']
            if sport == 'tennis':
                return kategorie['Tenisová hala']
            if sport == 'basketball':
                return kategorie['Basketbalová hala']
        if is_standalone('bazén', name):
            if sportovisko['location'] == 'indoor' or leisure == 'building' or sportovisko['building']: 
                return kategorie['Vnútorný bazén']
            return kategorie['Vonkajší bazén']
        if 'areál' in name:
            if leisure != 'water_park':
                return kategorie['Športový areál']
        if 'centrum' in name:
            return kategorie['Športové centrum']
        
    # Zimne
    if sport in ['ice_skating', 'ice_hockey'] or leisure == 'ice_rink':
        if sportovisko['building'] == 'yes':
            if leisure == 'ice_rink': return kategorie['Klzisko']
            return kategorie['Zimný štadión']
        if sportovisko['natural'] == 'water': return kategorie['Prírodný ľad']
        if leisure=='ice_rink': return kategorie['Vonkajšia ľadová plocha']
    
    # Plavanie
    if sport == 'swimming' or leisure == 'swimming_pool' or leisure=='water_park':
        if sportovisko['water'] in ['lake', 'reservoir'] or leisure == 'fishing': return kategorie['Jazero']
        if leisure == 'swimming_pool':
            if sportovisko['location'] == 'indoor' or leisure == 'building' or sportovisko['building']:
                return kategorie['Vnútorný bazén']
            else:
                return kategorie['Vonkajší bazén']
        if leisure == 'water_park' or sportovisko['landuse'] == 'recreation_ground':
            return kategorie['Kúpalisko']
        if leisure == 'swimming_area' or sportovisko['natural'] == 'beach':
            return kategorie['Pláž / Plavecká oblasť']
        if sportovisko['location'] == 'indoor' or leisure == 'building' or sportovisko['building']:
                return kategorie['Vnútorný bazén']
        return kategorie['Pláž / Plavecká oblasť']
        
    # default je outdoor
    if sportovisko['location'] == 'indoor' or sportovisko['building'] not in [None, 'stadium'] or leisure in ['sports_hall']:
        if leisure == 'sports_hall':
            if sport == 'multi' or len(sporty) > 1: return kategorie['Športová hala']
            if sport == 'basketball': return kategorie['Basketbalová hala']
            if sport == 'tennis': return kategorie['Tenisová hala']
        if leisure == 'sports_centre': return kategorie['Športové centrum']
    
    if sportovisko['landuse'] == 'meadow': return kategorie['Lúka']

    if leisure == 'sports_centre': return kategorie['Športový areál']
    if leisure in ['pitch', 'stadium']:
        if leisure == 'pitch':
            if {'multi'}.intersection(sporty) or len(sporty) > 1: return kategorie['Multifunkčné ihrisko']
            if sport == 'volleyball': return kategorie['Volejbalové ihrisko']
            if sport == 'beachvolleyball': return kategorie['Ihrisko na plážový volejbal']
            if sport in ['soccer', 'football']: return kategorie['Futbalové ihrisko']
            if sport == 'tennis': return kategorie['Tenisový kurt']
            if sport == 'basketball': return kategorie['Basketbalové ihrisko']
            if {'street_hockey', 'field_hockey', 'hockey'}.intersection(sporty): return kategorie['Hokejbalové ihrisko']
            return kategorie['Ihrisko (nešpecifikované)']
        if leisure == 'stadium':
            if {'ice_hockey','ice_skating'}.intersection(sporty) : return kategorie['Zimný štadión']
            if {'soccer', 'football'}.intersection(sporty): return kategorie['Futbalový štadión']
            return kategorie['Štadión (nešpecifikované)']
    
    return kategorie['Nezaradené']


In [13]:
classified_data['typ'] = classified_data.apply(lambda sportovisko: kategorizuj_sportovisko(sportovisko), axis = 1)
classified_data[['id','typ', 'nazov','sport', 'leisure', 'vsetky_sporty_str','vsetky_sporty', 'suradnice']].head(15)

,id,typ,nazov,sport,leisure,vsetky_sporty_str,vsetky_sporty,suradnice
0,relation/9105793,1230,None,None,stadium,,{},POINT (17.59181683837953 48.37372144153096)
1,relation/9108227,1230,Štadión MŠK Žilina,None,stadium,,{},POINT (18.74480877435066 49.22912718944628)
2,way/4254858,1140,None,soccer,pitch,soccer,{soccer},POINT (17.29423143682134 48.19521065613523)
3,way/4264932,1210,FK Cepit Vajnory,soccer,sports_centre,soccer,{soccer},POINT (17.215580120434083 48.20993125832007)
4,way/4351142,3230,Vodná nádrž Kurinec,swimming,swimming_area,swimming,{swimming},POINT (20.018764931862275 48.34455124823878)
5,way/4423886,1140,None,soccer,pitch,soccer,{soccer},POINT (17.21904810546658 48.25030266601612)
6,way/4514167,1210,FK Rača,multi,pitch,multi,{multi},POINT (17.14752169308766 48.201524398053614)
7,way/4692275,3220,Aquapark Senec,None,water_park,,{},POINT (17.414970668830268 48.22180974465234)
8,way/4815784,1140,None,soccer,pitch,soccer,{soccer},POINT (20.218243116409436 49.13625139972391)
9,way/4907049,1320,Devínskonovoveský rybník,ice_skating,fishing,ice_skating,{ice_skating},POINT (16.97293964321636 48.216569082274624)


In [14]:
# pripad, ktory predtym nemal indoor atribut, ale dedenim ho ziskal a bol preto presne klasifikovany
classified_data[classified_data['nazov'] == 'Vírivý bazén malý']['typ'] == 3110 # Vnutorny bazen

7281    True
Name: typ, dtype: bool

In [15]:
chyba_kategoria = classified_data[classified_data['typ'] == 0]
chyba_kategoria

,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,kontakt_webstranka,max_hlbka,min_hlbka,hlbka,teplota_vody,nazov,max_teplota,kapacita_divaci,poplatok,typ
449,way/58813782,way/58813782,None,None,None,None,None,Kozárovce,928,SK,...,None,None,None,None,None,None,None,None,None,0
633,way/71530655,way/71530655,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
1006,way/111579168,way/111579168,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
1322,way/143846370,way/143846370,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
3678,way/327819364,way/327819364,None,None,None,None,None,Vysoká pri Morave,781,SK,...,None,None,None,None,None,None,None,None,None,0
3714,way/330437267,way/330437267,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
4639,way/449575295,way/449575295,None,None,None,customers,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
9100,way/1052855043,way/1052855043,None,None,None,None,None,Prešov,15170,SK,...,https://www.profitennisacademy.sk/,None,None,None,None,Profitennis Academy,None,None,None,0
11407,node/923003916,node/923003916,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
11418,node/1230256716,node/1230256716,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0


In [16]:
#classified_data = classified_data[classified_data['typ'] != 0]

ĎALŠÍ PREPROCESSING DÁT

In [17]:
def da_sa_hrat_futbal(sportovisko):
    """
    Prepočíta, či sa dá na športovisku hrať futbal.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak sa dá na športovisku hrať futbal, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 1140, "vsetky_sporty": {"soccer", "basketball"}})
    >>> da_sa_hrat_futbal(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Futbalové ihrisko'], kategorie['Futbalový štadión']] or {'soccer', 'football'}.intersection(sporty):
        return 1
    return 0

def da_sa_hrat_basketbal(sportovisko):
    """
    Prepočíta, či sa dá na športovisku hrať basketbal.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak sa dá na športovisku hrať basketbal, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 1160, "vsetky_sporty": {"basketball", "tennis"}})
    >>> da_sa_hrat_basketbal(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Basketbalové ihrisko'], kategorie['Basketbalová hala']] or {'basketball'}.intersection(sporty):
        return 1
    return 0

def da_sa_hrat_volejbal(sportovisko):
    """
    Prepočíta, či sa dá na športovisku hrať volejbal.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak sa dá na športovisku hrať volejbal, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 1120, "vsetky_sporty": {"volleyball", "beachvolleyball"}})
    >>> da_sa_hrat_volejbal(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Volejbalové ihrisko'], kategorie['Ihrisko na plážový volejbal']] or {'volleyball', 'beachvolleyball'}.intersection(sporty):
        return 1
    return 0

def da_sa_hrat_tenis(sportovisko):
    """
    Prepočíta, či sa dá na športovisku hrať tenis.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak sa dá na športovisku hrať tenis, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 1150, "vsetky_sporty": {"tennis"}})
    >>> da_sa_hrat_tenis(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Tenisová hala'], kategorie['Tenisový kurt']] or {'tennis'}.intersection(sporty):
        return 1
    return 0

def da_sa_hrat_hokej(sportovisko):
    """
    Prepočíta, či sa dá na športovisku hrať hokej.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak sa dá na športovisku hrať hokej, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 1310, "vsetky_sporty": {"ice_hockey", "street_hockey"}})
    >>> da_sa_hrat_hokej(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Vonkajšia ľadová plocha'], kategorie['Prírodný ľad'], kategorie['Zimný štadión'], kategorie['Hokejová hala'], kategorie['Hokejbalové ihrisko'], kategorie['Hokejbalová hala']] or {'ice_hockey', 'street_hockey', 'field_hockey'}.intersection(sporty):
        return 1
    return 0

def da_sa_plavat(sportovisko):
    """
    Prepočíta, či sa dá na športovisku plávať.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak sa dá na športovisku plávať, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 3110, "vsetky_sporty": {"swimming"}})
    >>> da_sa_plavat(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Vnútorný bazén'], kategorie['Vonkajší bazén'], kategorie['Jazero'], kategorie['Plaváreň'], kategorie['Kúpalisko'], kategorie['Pláž / Plavecká oblasť']] or {'swimming'}.intersection(sporty):
        return 1
    return 0

def je_viacucelove(sportovisko):
    """
    Prepočíta, či je športovisko viacúčelové.

    Argumenty:
    sportovisko (pd.Series): Riadok DataFrame reprezentujúci športovisko.

    Návratová hodnota:
    int: 1 ak je športovisko viacúčelové, inak 0.

    Príklad použitia:
    >>> sportovisko = pd.Series({"typ": 1110, "vsetky_sporty": {"soccer", "tennis"}})
    >>> je_viacucelove(sportovisko)
    1
    """
    typ = sportovisko['typ']
    sporty = sportovisko['vsetky_sporty']
    if typ in [kategorie['Multifunkčné ihrisko'], kategorie['Športová hala'], kategorie['Športový areál'], kategorie['Športové centrum'], kategorie['Telocvičňa']] or {'multi'}.intersection(sporty) or len(sporty) > 1:
        return 1
    return 0


In [18]:
try:
    classified_data = classified_data.drop(['futbal', 'hokej', 'basketbal', 'plavanie', 'tenis', 'volejbal', 'multi'], axis = 1)
except:
    classified_data = classified_data.drop(['futbal', 'hokej', 'basketbal', 'plavanie', 'tenis', 'volejbal', 'viacucelove'], axis = 1)

classified_data['futbal'] = classified_data.apply(da_sa_hrat_futbal, axis = 1)
classified_data['tenis'] = classified_data.apply(da_sa_hrat_tenis, axis = 1)
classified_data['basketbal'] = classified_data.apply(da_sa_hrat_basketbal, axis = 1)
classified_data['hokej'] = classified_data.apply(da_sa_hrat_hokej, axis = 1)
classified_data['volejbal'] = classified_data.apply(da_sa_hrat_volejbal, axis = 1)
classified_data['plavanie'] = classified_data.apply(da_sa_plavat, axis = 1)
classified_data['viacucelove'] = classified_data.apply(je_viacucelove, axis = 1)
classified_data['zdroj'] = 'OpenStreetMap'
classified_data['status'] = classified_data.apply(lambda x: 'Aktívne' if x['abandoned'] is None else 'Neaktívne', axis = 1)

classified_data

,id,@id,FIXME,abandoned,abandoned:leisure,access,access:conditional,addr:city,addr:conscriptionnumber,addr:country,...,typ,futbal,tenis,basketbal,hokej,volejbal,plavanie,viacucelove,zdroj,status
0,relation/9105793,relation/9105793,None,None,None,None,None,None,None,None,...,1230,0,0,0,0,0,0,0,OpenStreetMap,Aktívne
1,relation/9108227,relation/9108227,None,None,None,None,None,None,None,None,...,1230,0,0,0,0,0,0,0,OpenStreetMap,Aktívne
2,way/4254858,way/4254858,None,None,None,None,None,None,None,None,...,1140,1,0,0,0,0,0,0,OpenStreetMap,Aktívne
3,way/4264932,way/4264932,None,None,None,None,None,None,None,None,...,1210,1,0,0,0,0,0,0,OpenStreetMap,Aktívne
4,way/4351142,way/4351142,None,None,None,yes,None,None,None,None,...,3230,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,node/11847211830,node/11847211830,None,None,None,private,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12572,node/11847211831,node/11847211831,None,None,None,private,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12573,node/11847211846,node/11847211846,None,None,None,private,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12574,node/11847352225,node/11847352225,None,None,None,private,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne


In [19]:
# Vyhodi nepotrebne atributy
columns_to_drop = [
    '@id',
    'CLC:code',
    'CLC:shapeId',
    'FIXME',
    'abandoned:leisure',
    'access:conditional',
    'addr:country',
    'addr:suburb',
    'addr:unit',
    'architect',
    'barrier',
    'bicycle',
    'brand',
    'building:architecture',
    'building:colour',
    'building:levels',
    'building:material',
    'check_date:lit',
    'check_date:surface',
    'club',
    'colour',
    'community_centre',
    'community_centre:for',
    'complete',
    'created_by',
    'currency:EUR',
    'descent',
    'description:en',
    'description:hu',
    'designation',
    'disused',
    'disused:club',
    'disused:leisure',
    'ele',
    'elevation',
    'fax',
    'fishing',
    'fixme',
    'highway',
    'import',
    'import_ref',
    'inline_skates',
    'internet_access:fee',
    'isced:level',
    'layer',
    'level',
    'mapillary',
    'massage',
    'material',
    'max_age',
    'maxstay',
    'min_age',
    'name:cs',
    'name:de',
    'name:en',
    'name:etymology:wikidata',
    'name:hu',
    'name:it',
    'name:pl',
    'noname',
    'nudism',
    'old_name',
    'opening_hours:covid19',
    'operator:wikidata',
    'payment:cash',
    'payment:coins',
    'piste:type',
    'playground:theme',
    'postal_code',
    'recommended_age',
    'ref',
    'ref:minvskaddress',
    'ref:vatin',
    'roof:colour',
    'roof:height',
    'roof:levels',
    'roof:orientation',
    'roof:shape',
    'sazp:type',
    'service',
    'service:bicycle:rental',
    'shop',
    'short_name',
    'smoothness',
    'source:access',
    'source:addr',
    'source:conscriptionnumber',
    'source:geometry',
    'source:name',
    'source:shape',
    'source:street',
    'source:streetnumber',
    'sport_1',
    'sport_2',
    'sport_3',
    'type',
    'urbanatlas:CITIES',
    'urbanatlas:CODE',
    'urbanatlas:LUZ_OR_CIT',
    'urbanatlas:PROD_DATE',
    '@relations',
    'wikidata',
    'wikimedia_commons',
    'wikipedia',
    'validne_sporty',
    'motor_vehicle',
    'addr:city',
'addr:conscriptionnumber',
'addr:floor',
'addr:housename',
'addr:housenumber',
'addr:place',
'bath:type',
'asphalt',
'bath:type',
'description',
'dog',
'image',
'landcover',
'note',
'description',
'tourism',
'fence_type',
'outdoor_seating',
'payment:credit_cards',
'payment:debit_cards',
'sauna',
'shower',
'smoking',
'@version',
'@changeset',
'@user',
'@uid',	
'@relations'
]

try:
    for column_name in columns_to_drop:
        if column_name in classified_data.columns:
            classified_data = classified_data.drop(column_name, axis = 1)
except Exception as e:
    print('uz vymazane', e)

classified_data

,id,abandoned,access,addr:postcode,addr:street,addr:streetnumber,amenity,building,contact:instagram,contact:tiktok,...,typ,futbal,tenis,basketbal,hokej,volejbal,plavanie,viacucelove,zdroj,status
0,relation/9105793,None,None,None,None,None,None,stadium,None,None,...,1230,0,0,0,0,0,0,0,OpenStreetMap,Aktívne
1,relation/9108227,None,None,010 01,None,None,None,stadium,None,None,...,1230,0,0,0,0,0,0,0,OpenStreetMap,Aktívne
2,way/4254858,None,None,None,None,None,None,None,None,None,...,1140,1,0,0,0,0,0,0,OpenStreetMap,Aktívne
3,way/4264932,None,None,None,None,None,None,None,None,None,...,1210,1,0,0,0,0,0,0,OpenStreetMap,Aktívne
4,way/4351142,None,yes,None,None,None,None,None,None,None,...,3230,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,node/11847211830,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12572,node/11847211831,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12573,node/11847211846,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12574,node/11847352225,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne


In [20]:
classified_data.columns

Index(['id', 'abandoned', 'access', 'addr:postcode', 'addr:street',
       'addr:streetnumber', 'amenity', 'building', 'contact:instagram',
       'contact:tiktok', 'contact:youtube', 'drinking_water', 'height',
       'hoops', 'internet_access', 'landuse', 'leisure', 'leisure_1', 'length',
       'lit', 'location', 'natural', 'net', 'opening_hours', 'operator',
       'operator:type', 'owner', 'reservation', 'seasonal', 'source', 'sport',
       'start_date', 'supervised', 'surface', 'swimming_lanes',
       'swimming_pool', 'toilets:wheelchair', 'water', 'water:taste',
       'water_characteristic', 'wheelchair', 'width', '@timestamp', 'geometry',
       'vsetky_sporty', 'vsetky_sporty_str', 'suradnice', 'kontakt_email',
       'kontakt_facebook', 'kontakt_telefon', 'kontakt_webstranka',
       'max_hlbka', 'min_hlbka', 'hlbka', 'teplota_vody', 'nazov',
       'max_teplota', 'kapacita_divaci', 'poplatok', 'typ', 'futbal', 'tenis',
       'basketbal', 'hokej', 'volejbal', 'plavanie', 

In [21]:
# Sportovisko, ktore obsahuje ine sportoviska, zdedi binarne priznaky ukazujuce prispobenie nejakemu sportu od sportovisk, ktore obsauje tak, ze mu bude pridelene maximum z danych dvoch hodnot
# napr. s1 obsahuje s2 a s1[futbal] = 0, ale s2[futbal] = 1, v tomto pripade max(s1[futbal], s2[futbal]) = s2[futbal] a teda s1 zdedi informaciu o moznosti hrat futbal.

inheritance_data = classified_data.copy()
joined_gdf = gpd.sjoin(inheritance_data, inheritance_data, how="inner", predicate="contains")
columns_to_inherit = ['futbal', 'tenis', 'basketbal', 'hokej', 'volejbal', 'plavanie', 'viacucelove']
for column in columns_to_inherit:
    inherited_data = joined_gdf.groupby(joined_gdf.index)[column + '_right'].max()

    inheritance_data.loc[inherited_data.index, column + '_inherited'] = inherited_data
    classified_data.loc[inherited_data.index, column] = inherited_data

In [22]:
classified_data

,id,abandoned,access,addr:postcode,addr:street,addr:streetnumber,amenity,building,contact:instagram,contact:tiktok,...,typ,futbal,tenis,basketbal,hokej,volejbal,plavanie,viacucelove,zdroj,status
0,relation/9105793,None,None,None,None,None,None,stadium,None,None,...,1230,0,0,0,0,0,0,0,OpenStreetMap,Aktívne
1,relation/9108227,None,None,010 01,None,None,None,stadium,None,None,...,1230,0,0,0,0,0,0,0,OpenStreetMap,Aktívne
2,way/4254858,None,None,None,None,None,None,None,None,None,...,1140,1,0,0,0,0,0,0,OpenStreetMap,Aktívne
3,way/4264932,None,None,None,None,None,None,None,None,None,...,1210,1,0,0,0,0,0,0,OpenStreetMap,Aktívne
4,way/4351142,None,yes,None,None,None,None,None,None,None,...,3230,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,node/11847211830,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12572,node/11847211831,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12573,node/11847211846,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne
12574,node/11847352225,None,private,None,None,None,None,None,None,None,...,3210,0,0,0,0,0,1,0,OpenStreetMap,Aktívne


In [23]:
classified_data.columns

Index(['id', 'abandoned', 'access', 'addr:postcode', 'addr:street',
       'addr:streetnumber', 'amenity', 'building', 'contact:instagram',
       'contact:tiktok', 'contact:youtube', 'drinking_water', 'height',
       'hoops', 'internet_access', 'landuse', 'leisure', 'leisure_1', 'length',
       'lit', 'location', 'natural', 'net', 'opening_hours', 'operator',
       'operator:type', 'owner', 'reservation', 'seasonal', 'source', 'sport',
       'start_date', 'supervised', 'surface', 'swimming_lanes',
       'swimming_pool', 'toilets:wheelchair', 'water', 'water:taste',
       'water_characteristic', 'wheelchair', 'width', '@timestamp', 'geometry',
       'vsetky_sporty', 'vsetky_sporty_str', 'suradnice', 'kontakt_email',
       'kontakt_facebook', 'kontakt_telefon', 'kontakt_webstranka',
       'max_hlbka', 'min_hlbka', 'hlbka', 'teplota_vody', 'nazov',
       'max_teplota', 'kapacita_divaci', 'poplatok', 'typ', 'futbal', 'tenis',
       'basketbal', 'hokej', 'volejbal', 'plavanie', 

EXPORT

In [24]:
# Upravy pred exportom a export
final_data = classified_data.drop(['suradnice','vsetky_sporty'], axis = 1)

final_data.to_file(r'./source-files/1_klasifikovane.geojson', driver = 'GeoJSON')